# Test place for everything

# import functions

In [1]:
from sqlays import export_sql, import_sql
from matheq import Cel_to_Kel, Kel_to_Cel, to_pot_temp
from mapscre import station_map, bathy_data, contour_ver, TS_diagram
from iscays import sum_up, vertical_plot, depth_bin_interval, particle_bin_interval, isc_xlsx 
from ctdays import find_start_row, raw_ctd_to_df
from graphcre import deployment_constancy, vertical_each_org_distribution, stacked_vertical_distribution, comp_vertical_distribution 
from matplotlib import pyplot as plt
import pandas as pd
import os, glob

# extract data and image from LOKI_browser by certain interval (distance)

In [ ]:
proj_dir = ('') # dir of folder extracted from loki_browser
meta_df =  glob.glob(os.path.join(proj_dir,'*.csv')) # glob meta data to get the info

image_datetime = meta_df['Image']



# particle VS zooplankton

In [2]:
file_path = os.path.expanduser('~/Git/OCEANpy/data')
loki_df = export_sql('loki', ['ps107'], 'awi_server')
isc_stb = export_sql('isc', 'isc_meta', 'local')
isc_file_list = glob.glob(file_path+os.sep+'isc'+os.sep+'*.xlsx')
isc_each_df = isc_stb[isc_stb['cruise_station_haul'].str.contains('PS107') & isc_stb['profile_number'].notnull()] # select based on the cruise


In [11]:
for index, value in isc_each_df.iterrows():
    # select one station from one isc station and get cruise name, station number and profilenumber of ISC
    cruise, station, profile_num = value['cruise_station_haul'].split('_')[0], int(float(value['cruise_station_haul'].split('_')[1])), int(float(value['profile_number']))
    
    # Based on the cruise and profilenumber of ISC import data from excel file
    isc_excel = str('IR'+str(profile_num).zfill(2)+'_DownCast_'+cruise+'.xlsx')
    for i in isc_file_list:
        if i.split(os.sep)[-1] == isc_excel:
            particle_range = [150, 500, 1000, 100000]
            
            # from here specific station.
            isc_ctd_df, vol_spec_df, aggr_con_df, size_spec_df = isc_xlsx(i, 10, particle_range)
            vol, abd, isc_ctd, ctd_dict = vertical_plot(isc_ctd_df, vol_spec_df, aggr_con_df, 'delete', particle_range)
            loki_station_df = loki_df[loki_df['station'].str.contains(cruise+'_'+str(station).zfill(3))]
            if loki_station_df.empty:
                continue
            else:
                copepoda_df = loki_station_df[loki_station_df['copepod']==True]
                copepoda_depth = copepoda_df['loki_depth'].to_list()
                
                fig, axs = plt.subplots(3,3, figsize = (10, 15))
                axs = axs.ravel()

                # 1st plot for temperature and salanity / data from ISC CTD
                axs[0].plot(isc_ctd['temp'], isc_ctd['depth'], color='red', linewidth= 1, alpha=0.7)
                axs[0].set_ylabel(ctd_dict['depth'], color ='black')
                axs[0].set_xlabel(ctd_dict['temp'], color = 'red')
                axs[0].set_xlim(-3, 9)
                axs[0].set_ylim(0,500)
                axs[0].invert_yaxis()


                sec_axs = axs[0].twiny()
                sec_axs.plot(isc_ctd['sal'], isc_ctd['depth'], color ='blue', linewidth= 1, alpha=0.7)
                sec_axs.set_xlabel(ctd_dict['sal'], color = 'blue')
                sec_axs.set_xlim(30, 36)

                # 2nd plot for turbidity and Fluorescence / data from ISC CTD
                axs[3].plot(isc_ctd['turb'], isc_ctd['depth'], color='red', linewidth= 1, alpha=0.7)
                axs[3].set_ylabel(ctd_dict['depth'], color ='black')
                axs[3].set_xlabel(ctd_dict['turb'], color = 'red')
                axs[3].set_xlim(-0.5, 6)
                axs[3].set_ylim(0,500)
                axs[3].invert_yaxis()


                sec_axs = axs[3].twiny()
                sec_axs.plot(isc_ctd['fluo'], isc_ctd['depth'], color ='blue', linewidth= 1, alpha=0.7)
                sec_axs.set_xlabel(ctd_dict['fluo'], color = 'blue')
                sec_axs.set_xlim(-0.5, 6)

                # 3rd plot for Total Aggregate Volume and Aggregate abundance
                axs[1].plot(vol['vol_sml'], isc_ctd['depth'], color='red', linewidth= 1, alpha=0.7)
                axs[1].set_xlabel('vol 0.15-0.5 [mm]', color = 'red')
                axs[1].set_ylim(0,500)
                axs[1].invert_yaxis()

                sec_axs = axs[1].twiny()
                sec_axs.plot(vol['vol_med'], isc_ctd['depth'], color ='blue', linewidth= 1, alpha=0.7)
                sec_axs.set_xlabel('vol 0.5-1 [mm]', color = 'blue')

                # 4th plot for vol_median ESD and Average ESD
                axs[4].plot(vol['vol_lrg'], isc_ctd['depth'], color='red', linewidth= 1, alpha=0.7)
                axs[4].set_xlabel('vol 1 [mm] > ', color = 'red')
                axs[4].set_ylim(0,500)
                axs[4].invert_yaxis()

                sec_axs = axs[4].twiny()
                sec_axs.plot(vol['vol_tol'], isc_ctd['depth'], color ='blue', linewidth= 1, alpha=0.7)
                sec_axs.set_xlabel('vol total (0.15 [mm] > )', color = 'blue')

                # 5th plot for Total Aggregate Volume and Aggregate abundance
                axs[2].plot(abd['abd_sml'], isc_ctd['depth'], color='red', linewidth= 1, alpha=0.7)
                axs[2].set_xlabel('abd 0.15-0.5 [mm]', color = 'red')
                axs[2].set_ylim(0,500)
                axs[2].invert_yaxis()

                sec_axs = axs[2].twiny()
                sec_axs.plot(abd['abd_med'], isc_ctd['depth'], color ='blue', linewidth= 1, alpha=0.7)
                sec_axs.set_xlabel('abd 0.5-1 [mm]', color = 'blue')

                # 6th plot for vol_median ESD and Average ESD
                axs[5].plot(abd['abd_lrg'], isc_ctd['depth'], color='red', linewidth= 1, alpha=0.7)
                axs[5].set_xlabel('abd 1 [mm] > ', color = 'red')
                axs[5].set_ylim(0,500)
                axs[5].invert_yaxis()

                sec_axs = axs[5].twiny()
                sec_axs.plot(abd['abd_tol'], isc_ctd['depth'], color ='blue', linewidth= 1, alpha=0.7)
                sec_axs.set_xlabel('abd total (0.15 [mm] > )', color = 'blue')
                
                # plot for zooplankton
                axs[6].hist(copepoda_depth, orientation='horizontal', bins = int(max(copepoda_depth)/10), color ='black')
                axs[6].set_ylim(0,500)
                axs[6].set_xlabel('copepoda [count]', color = 'black')
                axs[6].invert_yaxis()
                

                # addtional plot modification
                fig.tight_layout(pad=3, h_pad=1, w_pad=1) # adjust layout of subplots
                #plt.suptitle(title, y = 0.99) # main title
                
                os.chdir(os.path.expanduser('~/Git/OCEANpy/plots'))
                fig_name = str('isc_loki_comp'+cruise+'_'+str(station)+'.png')
                plt.savefig(fig_name)
                plt.close()
                

# create TS diagram

In [ ]:
ctd_df = export_sql('ctd', 'ctd_meta')
#station_list = ['PS107_18']
#ctd_df = ctd_df.loc[ctd_df['Event'].str.contains('|'.join(station_list))]
ctd_df = ctd_df.loc[pd.to_numeric(ctd_df['Depth water [m]']) <=1000]
depth = tuple(pd.to_numeric(ctd_df['Depth water [m]']))
sal = tuple(pd.to_numeric(ctd_df['Sal']))
temp = tuple(pd.to_numeric(ctd_df['Temp [°C]']))
TSD_dict = {'temp': temp, 'sal': sal, 'depth':depth}
TS_diagram(TSD_dict)


# create contour plot

In [ ]:
ctd_df = export_sql('ctd', 'ctd_meta'); print(list(ctd_df))
station_list = ['PS107_10', 'PS107_12', 'PS107_14', 'PS107_16', 'PS107_18']
cols_to_use = ['Latitude', 'Longitude', 'Depth water [m]', 'Press [dbar]']
ctd_df_filter = ctd_df.loc[ctd_df['Event'].str.contains('|'.join(station_list))] # select by station
#ctd_df_filter = ctd_df_filter[cols_to_use] # select only necessary columns
ctd_df_filter = ctd_df.loc[(pd.to_numeric(ctd_df['Latitude']) >=78.9) & (pd.to_numeric(ctd_df['Latitude']) <=79.1)] # select by lat and lon
ctd_df_filter = ctd_df_filter.loc[(pd.to_numeric(ctd_df_filter['Depth water [m]']) <=1000)]
ctd_df_filter = ctd_df_filter.dropna() # drop rows having empty value (empty values will affect contouring method)

ctd_df_filter['d_n'] = ctd_df_filter.apply(lambda row: day_night(float(row['Latitude']), float(row['Longitude']), datetime.strptime(row['Date/Time'], '%Y-%m-%dT%H:%M')), axis=1 )
ctd_df_filter = ctd_df_filter.loc[ctd_df_filter['d_n'] == 'day']

depth = [i*(-1) for i in list(pd.to_numeric(ctd_df_filter['Depth water [m]']).values)]
lat = list(pd.to_numeric(ctd_df_filter['Latitude']).values)
lon = list(pd.to_numeric(ctd_df_filter['Longitude']).values)
z = list(pd.to_numeric(ctd_df_filter['Press [dbar]']).values)

ctd_array = np.array((lat, lon, z, depth)).transpose() # create array

topo = bathy_data (75, 83, -30, 20)
contour_ver(topo, 'lat', 79, (-5, 10), ctd_array)
#contour_ver(topo, 'lon', 79, (-5, 10), ctd_array)





# create station map

In [ ]:
isc_df = export_sql('isc', 'isc_meta')
cruise_list = ('PS107', 'PS114', 'PS121')
cruise_dict = {}
for cruise in cruise_list:
    cruise_df = isc_df[isc_df['cruise_station_haul'].str.contains(cruise)]
    cruise_dict[cruise] = (tuple(pd.to_numeric(cruise_df['lat'])) , tuple(pd.to_numeric(cruise_df['lon'])))

topo = bathy_data (75, 83, -30, 20) # -30, 30, 75, 85
station_map(cruise_dict, topo)

# get data from SQL

In [ ]:
action = 'export'
if action == 'export':
    edit = ['ps107'] # cruise name (in list, case insensitive) for AWI server and table name for local
    df = export_sql('loki', edit, 'awi_server')
    print(df)

elif action == 'import':
    filepath = os.path.expanduser('~/Git/OCEANpy/data/loki/LOKI_station.xlsx')
    df = pd.read_excel(filepath)
    # df = raw_ctd_to_df(filepath) # processing for raw ctd file
    import_sql('loki', 'loki_meta', df, 'replace')

# ISC data

In [ ]:
file_path = os.path.expanduser('~/Git/OCEANpy')
for excel_file in glob.glob(file_path+os.sep+'data'+os.sep+'isc'+os.sep+'IR*.xlsx'):
    particle_range = [150, 500, 1000, 100000]
    ctd_df, vol_spec_df, aggr_con_df, size_spec_df = isc_xlsx(excel_file, 10, particle_range)
    vertical_plot(ctd_df, vol_spec_df, aggr_con_df, excel_file.split(os.sep)[-1].replace('.xlsx', ''), particle_range)